# GETi Model Management

This notebook shows how to work with models, algorithms, model groups and optimized models in a GETi project. The Geti SDK provides a `ModelClient` class, that allows to fetch a list of models and model details for a specific project, set an active model or even start an optimization job for a model. This provides a Python interface for model management in a GETi project and helps creating an optimal pipeline for the task.

In [ ]:
# As usual we will connect to the platform first, using the server details from the .env file

from geti_sdk import Geti
from geti_sdk.rest_clients.project_client.project_client import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

# We will also create a ProjectClient for the server
project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

Specify the name of the project to download. We will use the `COCO multitask animal demo` project created in notebook [004](004_create_pipeline_project_from_dataset.ipynb).\
We instantiate a Project Client to retrieve the project and examine the tasks that it contains.

In [ ]:
from geti_sdk.demos import ensure_trained_example_project

PROJECT_NAME = "COCO multitask animal demo"
ensure_trained_example_project(geti=geti, project_name=PROJECT_NAME)


project = project_client.get_project_by_name("COCO multitask animal demo")
trainable_tasks = project.get_trainable_tasks()
for task in trainable_tasks:
    print(task.summary)

Let's explore the models behind the pipeline in our project. We will need a Model Client to do that.

In [ ]:
from geti_sdk.rest_clients.model_client import ModelClient

model_client = ModelClient(
    workspace_id=geti.workspace_id, project=project, session=geti.session
)

In [ ]:
active_models = model_client.get_all_active_models()
print("Active models:")
for i, model in enumerate(active_models):
    print(f"Task {i + 1}: ", model.name)

## Choose from supported algorithms for the project.
In GETi terminology, each model implements an algorithm. An **algorithm** serves as a blueprint for a model, defining its architecture.\
The **model** consists of weights and can be used for making predictions. Every model was trained with specific hyperparameters on a specific dataset and also possibly was optimized.\
By using a Training Client, we can explore available algorithms for the project and select the one that best suits our purpose.

In [ ]:
from typing import List

from geti_sdk.data_models.containers.algorithm_list import AlgorithmList
from geti_sdk.rest_clients.training_client import TrainingClient

training_client = TrainingClient(
    workspace_id=geti.workspace_id, project=project, session=geti.session
)
algorithms: List[AlgorithmList] = []
for task in trainable_tasks:
    algo_list_for_task = training_client.get_algorithms_for_task(task)
    algorithms.append(algo_list_for_task)
    default_algo_name = algo_list_for_task.get_default_for_task_type(
        task.task_type
    ).name
    print(f"Default algorithm for {task.title} is {default_algo_name}\n")
    print("Other available algorithms are")
    print(algo_list_for_task[:4].summary)

### Training a Model for an Algorithm
If the project was trained with active learning system and was not modified, the default algorithms match the models in the project.
However, we can train a new model for a heavier algorithm if we want more precision, or a lighter one if we need to increase throughput.

In the next few cells we will train a new model for the first task (detection) in the pipeline.
A user can also manipulate training hyperparameters by passing a *Taks Configuration* object. We will use this feature to change the default batch size.

In [ ]:
# Choose a task and algorithm we will use for training
task_number = 0
task_to_train = trainable_tasks[task_number]
algo_to_train = algorithms[task_number][0]

In [ ]:
from geti_sdk.rest_clients.configuration_client import ConfigurationClient

# Get the default task configuration from the server
configuration_client = ConfigurationClient(
    workspace_id=geti.workspace_id, project=project, session=geti.session
)
algorithm_hyperparameters = configuration_client.get_for_task_and_algorithm(
    task_to_train, algo_to_train
)

In [ ]:
# Update a hyperparameter and make sure the change is reflected in the configuration object
algorithm_hyperparameters.set_parameter_value(parameter_name="batch_size", value=32)
print(algorithm_hyperparameters.summary)

Now we are ready to train a new model for the choosen algorithm and hyperparameters. We will use the Training Client to start a job.

It may take about 10 minutes for the job to complete!

In [ ]:
print(f"Training {task_to_train.title} with {algo_to_train.name} algorithm\n")
job = training_client.train_task(
    algorithm=algo_to_train,
    task=task_to_train,
    hyper_parameters=algorithm_hyperparameters,
)
training_client.monitor_job(job);

Now we can examine the active models for the project again to see the newly trained model automaticaly set by the server as the active model.

In [ ]:
active_models = model_client.get_all_active_models()
print("Active models:")
for i, model in enumerate(active_models):
    print(f"Task {i + 1}: ", model.name)

### Optimizing a Model
If we want to increase the model's throughput or try out different precision levels, we can optimize the model using [OpenVINO Post Training Optimization feature](https://docs.openvino.ai/2024/openvino-workflow/model-optimization-guide/quantizing-models-post-training.html).\
We will use the Model Client to start an optimization job.

In [ ]:
job = model_client.optimize_model(model=active_models[0], optimization_type="pot")
_ = model_client.monitor_job(job)

## Work with model groups
Model Group is another entity in GETi that embraces all the trained models for a specific algorithm and task, differentiating them by version name. Working with model groups allows to access different model versions and switch between them if needed, seting them active at the Platform server.

The Model Client allows exploring model groups that are present in a project. If a model for some algorithm was trained for any task, the corresponding model will exist in the project.\
Model Group actually operates models in a shortened form to increase SDK performance and lower server load, thus to retrieve the full model we need to make an additional call to Model Client.

> Note: you can reuse a model object from any of the previous steps, the model is retrieved using the Model Group further only for demonstration purposes.

In [ ]:
model_groups = model_client.get_all_model_groups()
print("Model groups:")
for i, model_group in enumerate(model_groups):
    print(f"{i + 1}. ", model_group.name)

Now we can restore a representation object for a specific model from the model group.

In [ ]:
model_summary = model_groups[1].models[0]
model = model_client.update_model_detail(model_summary)

Remember we performed an optimization job for the model? Now we can also retrieve the optimized model from the model object.

In [ ]:
optimized_model = model.get_optimized_model(optimization_type="pot")
optimized_model

We can also ask the server to return a model in a lowered precision format, which can be used for deployment on some edge devices.

In [ ]:
model.get_optimized_model(precision="FP16")

If we want to change the active model on the server side, we can choose a model (or a model summary) from a desired model group and set it as active using the Model Client.

In [ ]:
model_client.set_active_model(model_groups[0].models[0])

## Model Deployment

We can also choose what model to use in a local deployment.\
We will create a Deployment Client and call the `deploy_project` method.

In [ ]:
from geti_sdk.rest_clients.deployment_client import DeploymentClient

deployment_client = DeploymentClient(
    workspace_id=geti.workspace_id, project=project, session=geti.session
)

The `deploy_project` method can accept a list of models for every task. In our case, we are ok with deploying the active model for the second task, so we will only pass a model for the first task.
For instance, we can use the optimized model from the previous step.

In [ ]:
deployment_client.deploy_project(models=[optimized_model])

Now, the deployment is ready for inference, you can refer to [008](008_deploy_project.ipynb) for more details on how to perform inference on the deployed project.

## Summary
In this notebook we made an impressive journey through the GETi model management capabilities. We explored the models, algorithms, model groups and optimized models in a GETi project. We trained a new model from a chosen algorithm for the detection task in the pipeline, optimized and deployed it. We also worked with model groups and discovered a way to navigate around different model versions in a project.